In [ ]:
import json
import re
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import pickle
import sklearn as sk
from yandex_geocoder import Client
client = Client("")


f = open('result.json', encoding= 'utf-8')
data = json.load(f)
hourofAttack = []
dayofAttack = []
methodUsed = []
locations = []
weekday = []

for i in data['messages']:
    dayofAttack.append(i['date'][:-9])
    for i in i['text']:
        r1 = re.findall(r"▶️\d{2}:\d{2}",str(i))
        r2 = re.findall(r"\d{3} мм", str(i), )
        r3 = re.findall(r"((н.п.|г.|г|н.п) ([^\s]+) (-|–) (н.п.|г.|г|н.п) ([^\s]+)( \(([^)]+)\))?)", str(i))
        if(r1 != [] and len(r1) == 1):
            hourofAttack.append(str(r1).replace('▶️', '')[2:-2]+':00')
        if(r2 != [] and len(r2) == 1):
            methodUsed.append(int(str(r2)[2:-5]))
        if(r3 != [] and len(r3) == 1):
            thing = str(r3[0][0])
            thing = re.split(' (-|–) ', thing)
            del(thing[1])
            origin = client.coordinates(thing[0])
            destination = client.coordinates(thing[1])
            locations.append([origin, destination])

print(locations)
with open('locations.pk', 'wb') as handle:
    pickle.dump(locations, handle, protocol=pickle.HIGHEST_PROTOCOL)

def renderHours(axes):
    df = pd.DataFrame(hourofAttack,  columns=['hour'])
    df['hour'] = pd.to_timedelta(df['hour'])
    df.set_index('hour', drop=False, inplace=True)
    ax = df.groupby(pd.Grouper(freq='1H')).count().plot(kind='bar', ax=axes[0,0])
    ax.set_xticks(range(0,24), range(0,24), rotation= 0)
    ax.set_xticklabels(range(0,24), fontsize=9)
    ax.set_xlabel('Hour of day')
    ax.set_ylabel('Incidents')
    [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 2 != 0]

def renderDays(axes):
    for i in dayofAttack:
        weekday.append(pd.to_datetime(i).strftime('%A'))

    cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    df = pd.DataFrame(weekday, columns=['day'])
    df['day'] = pd.Categorical(df['day'], categories=cats, ordered=True).sort_values('day')
    ax = df.groupby('day').value_counts().plot(kind='bar', ax=axes[1,1])
    ax.set_ylabel('Incidents')
    ax.set_xlabel('Day of Week')

def renderDate(axes):
    df = pd.DataFrame(dayofAttack, columns=['date'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', drop=False, inplace=True)
    ax = df.groupby(pd.Grouper(freq='1D')).count().rolling(10)['date'].mean().plot(ax=axes[0,1])
    ax.set_ylabel('Alerts')
    ax.set_xlabel('Date')
f.close()

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10))
plt.suptitle('Artillery strikes in Donetsk raion')
def renderMethods(axes):
    df = pd.DataFrame(methodUsed, columns=['methods']).sort_values('methods').groupby('methods').value_counts()
    ax = df[df.values > 10].plot(kind='bar', ax=axes[1,0])
    ax.set_ylabel('Incidents')
    ax.set_xlabel('Caliber')

renderHours(axes)
renderDays(axes)
renderDate(axes)
renderMethods(axes)